In [1]:
import random
import json
import pickle
import numpy as np

import nltk
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import load_model

In [2]:
lemmatizer = WordNetLemmatizer()

intents = json.loads(open("intents.json").read())

words = pickle.load(open("words.pkl","rb"))
classes = pickle.load(open("classes.pkl","rb"))
model = load_model("chatbot_model.h5")

In [3]:
classes

['age', 'goodbye', 'greetings', 'hours', 'name', 'shop']

In [16]:
def clean_sentence(sentences):
    sentence_words = nltk.word_tokenize(sentences)
    sentence_words = [lemmatizer.lemmatize(word) for word in sentence_words]
    return sentence_words

def bag_of_words(sentences):
    sentence_words = clean_sentence(sentences)
#     print(sentence_words)
    bag = [0]*len(words)
    for w in sentence_words:
        for i, word in enumerate(words):
#             print(word)
            if word == w:
                bag[i] = 1
    return np.array(bag)

def predict_class(sentences):
    bag = bag_of_words(sentences)
#     print(bag)
    res = model.predict(np.array([bag]))[0]
#     print(res)
    ERROR_THRESHOLD = 0.25
    result = [[i,r] for i,r in enumerate(res) if r> ERROR_THRESHOLD]
    result.sort(key=lambda x:x[1], reverse = True)
    return_list = []
    
    for r in result:
        return_list.append({'intent':classes[r[0]],'probability':str(r[1])})
    
    return return_list

def get_response(inents_list,intents_json):
    tag = inents_list[0]['intent']
    list_of_intents = intents_json['intents']
    
    for i in list_of_intents:
        if i['tag'] ==tag:
            result = random.choice(i['responses'])
            break
    return result

In [27]:
message = input("")
ints = predict_class(message)
res = get_response(ints,intents)
print(res)

hello
What can I do for you?


In [18]:
# ints = [{"intent":"greetings"}]
